# Import packages

In [1]:
# !pip install -U sentence-transformers

In [2]:
import re, os, random, time, math, json, pandas as pd, numpy as np
from datetime import date, datetime
import matplotlib.pyplot as plt, seaborn as sns
from tqdm import tqdm
from pyathena import connect
import boto3
from botocore.exceptions import ClientError
import openai
from openai import OpenAI
import pyarrow.parquet as pq
import tiktoken
import faiss
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
from chromadb.config import Settings

%matplotlib inline
sns.set_theme()

In [11]:
def convert_values(obj):
    """Recursively convert values in a nested structure."""
    if isinstance(obj, dict):
        return {k: convert_values(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_values(v) for v in obj]
    elif isinstance(obj, str):
        # Try to convert to int or float
        try:
            return int(obj)
        except ValueError:
            try:
                return float(obj)
            except ValueError:
                # Check if it's a date string (assuming YYYY-MM-DD format)
                try:
                    return date_to_int(obj)
                except ValueError:
                    return obj
    else:
        return obj

In [20]:
convert_values(json.loads({'publishdate': None}))

TypeError: the JSON object must be str, bytes or bytearray, not dict

# Functions and classes

## S3ParquetLoader & TpwireDataLoader

In [6]:
class S3ParquetLoader:
    def __init__(self, bucket_name, prefix, file_identifier="s3://"):
        self.bucket_name = bucket_name
        self.prefix = prefix
        self.file_identifier = file_identifier
        
    def load_s3_parquet(self, file):
        s3_path = self.file_identifier + self.bucket_name + self.prefix + file
        return pq.read_table(s3_path).to_pandas()

In [7]:
class TpwireDataLoader:
    def __init__(self, bucket_name, prefix, file_identifier="s3://"):
        self.s3_loader = S3ParquetLoader(bucket_name, prefix, file_identifier)
        
    def process_data(self, file_name):
        data = self.s3_loader.load_s3_parquet(file_name)
        # final_df = data[['summaryplaintext', 'publishdate', 'multi_story_flag', 'table_flag']]
        final_df = data
        final_df = final_df[(final_df['multi_story_flag'] == False) & (final_df['table_flag'] == False)]
        
        # tpwire_df = final_df[['summaryplaintext', 'publishdate']]
        tpwire_df = final_df.drop(['multi_story_flag', 'table_flag'], axis=1)
        tpwire_df = tpwire_df.sort_values(by='publishdate').reset_index(drop=True)
        tpwire_df['publishdate'] = tpwire_df['publishdate'].apply(lambda x: np.datetime64(x).astype(datetime).date()) # change to standard date format
        
        return tpwire_df

## IndexDoc

In [8]:
class IndexDoc:
    def __init__(self, embed):
        self.embed = embed
        
    def index_document(self, df):
        # for idx in range(len(df)):
        chunks = df['summaryplaintext'].tolist()

        index_tb = pd.DataFrame({
            'Content': chunks,
            'Vectors': embed.embed_documents(chunks), # 'original_index': range(len(chunks)),
        })

        index_tb.reset_index(drop=True, inplace=True)

        return index_tb

## tpwireDB

In [82]:
class tpwireDB:
    def __init__(self, client, name, openai_ef):
        self.client = client
        self.name = name
        self.embedding_function = openai_ef
        self.collection = self.get_or_create()
    
    # Collection methods
    def get_or_create(self):
        try:
            return self.client.get_or_create_collection(
                name=self.name,
                embedding_function=self.embedding_function
            )
        except Exception as e:
            print(f"Error getting or creating collection: {e}")
            return None
    
    def delete(self):
        try:
            self.client.delete_collection(name=self.name)
            print(f"Collection '{self.name}' deleted successfully.")
        except Exception as e:
            print(f"Error deleting collection: {e}")
    
    def collection_exists(self):
        try:
            collections = self.client.list_collections()
            return any(collection.name == self.name for collection in collections)
        except Exception as e:
            print(f"Error checking if collection exists: {e}")
            return False
       
    # Elements methods
    def document_exists(self, ids):
        try:
            result = self.collection.get(ids=ids)
            return [id in result['ids'] for id in ids]
        except Exception as e:
            print(f"Error checking document existence: {e}")
            return [False] * len(ids)

    
    def add(self, original_df):
        if self.collection is None:
            print("Collection not initialized. Cannot add data.")
            return
        
        # ids
        ids = [str(i) for i in original_df.index.tolist()] 
         
        # Check for existing IDs
        existing_ids = self.document_exists(ids)

        if any(existing_ids):
            existing_indices = [i for i, exists in enumerate(existing_ids) if exists]
            existing_id_values = [ids[i] for i in existing_indices]
            print(f"The following IDs already exist: {existing_id_values}. Cannot add data, try using upsert.")
            return

        # embeddings
        embeddings = [list(vec) for vec in original_df['Vectors'].values]

        # metadatas
        original_df = original_df.drop(['Content', 'Vectors'], axis=1)
        metadata_lst = []
        # for i, row in original_df.iterrows():
        #     meta_dict = {col: row[col][1:-1] if col != 'publishdate' else date_to_int(str(row[col])) for col in original_df.columns}
        #     metadata_lst.append(meta_dict)
        for i, row in original_df.iterrows():
            meta_dict = {
                'publishdate': date_to_int(
                    str(row['publishdate'])
                )
            }
            metadata_lst.append(meta_dict)
        

        self.collection.add(
            embeddings=embeddings,
            metadatas=metadata_lst,
            ids=ids
        )
    
    def upsert(self, new_df):
        if self.collection is None:
            print("Collection not initialized. Cannot upsert data.")
            return
        
        
        # ids
        ids = [str(i) for i in new_df.index.tolist()] 
        # embeddings
        embeddings = [list(vec) for vec in new_df['Vectors'].values]
        
        # metadatas
        new_df = new_df.drop(['Content', 'Vectors'], axis=1)
        metadata_lst = []
        # for i, row in new_df.iterrows():
        #     meta_dict = {col: row[col][1:-1] if col != 'publishdate' else date_to_int(str(row[col])) for col in new_df.columns}
        #     metadata_lst.append(meta_dict)
        
        for i, row in original_df.iterrows():
            meta_dict = {
                'publishdate': date_to_int(
                    str(row['publishdate'])
                )
            }
            metadata_lst.append(meta_dict)

        try:
            self.collection.upsert(
                embeddings=embeddings,
                metadatas=metadata_lst,
                ids=ids
            )
            print(f"Upserted {len(ids)} items in the collection.")
        except Exception as e:
            print(f"Error upserting data in collection: {e}")
        
    
    def retrieve(self, search_vectors, K, where=None):
        query_params = {
            "query_embeddings": search_vectors,
            "n_results": K,
            "include": ["distances"]
            # "include_metadata": True
        }
        
        if where:
            query_params["where"] = where # {"$and": [{"state": "NY"}, {"publishdate": {"$gte": date_to_int("2024-01-01")}}]}
    
        try:
            results = self.collection.query(**query_params)
            # results = self.collection.similarity_search_by_vector(**query_params)
            return results
        except Exception as e:
            print(f"Error retrieving data from collection: {e}")
            return None

## tpwireRAG

In [23]:
class tpwireRAG:
    def __init__(self, db, original_df, embed):
        self.db = db
        self.original_df = original_df
        self.embed = embed
        self.prompt_template = '''
        Use the following pieces of context to help you answer the question at the end. 

        If you don't know the answer, just output: "Answer": <I don't know the answer>. Don't try to make up an answer. 

        Keep the answer as concise as possible.

        Context: {context}
        Question: {search_text}
        
        "Answer": <answer> 
        ''' # Always say "thanks for asking!" at the end of the answer.
        self.metadata_prompt_template = """
        Please extract information from the given sentence and format it according to the ChromaDB filter standard. Follow these steps:

        1. Entity Extraction:
            - publishdate: Identify time-related keywords. If it is missing, return {{None}}.

        2. publishdate interpretation:
           - "recent" or "latest": Use date range from 30 days ago to today
           - "this quarter": Use range from start of current quarter to today
           - "in [month]": Use range from start to end of specified month
           - "in [month], [year]": Use range from start to end of specified month and year
           - Specific dates: Use as given
           - All dates should be should be in the standard yyyy-mm-dd format.

        3. Formatting:
           - If no publishdate is identified, return {{None}}.
           - Use the ChromaDB filter standard
           - For date ranges, use separate "$gte" and "$lte" conditions

        Examples:

        1. "Find office properties in New York City listed this quarter"
        {{
            "$and": [
                {{
                    "publishdate": {{"$gte": "2024-07-01"}}
                }},
                {{
                    "publishdate": {{"$lte": "2024-07-31"}}
                }}
            ]
        }}

        2. "Show recent retail listings in Los Angeles County, CA"
        {{
            "$and": [
                {{
                    "publishdate": {{"$gte": "2024-07-01"}}
                }},
                {{
                    "publishdate": {{"$lte": "2024-07-31"}}
                }}
            ]
        }}

        3. "What are the 5 transactions in New York in Dec, 2023?"
        {{
            "$and": [
                {{
                    "publishdate": {{"$gte": "2023-12-01"}}
                }},
                {{
                    "publishdate": {{"$lte": "2023-12-31"}}
                }}
            ]
        }}

        Please provide the ChromaDB filter for the following sentence:
        Input sentence: {sentence}
        """
    
    def retrieve_relevant(self, search_text, K): # , where=None
        response = openai_client.get_completion(message= self.metadata_prompt_template.format(sentence=search_text))
        where_filter = json.loads(response.choices[0].message.content)
        processed_filter = process_llm_output(where_filter)
        
        search_vectors = self.embed.embed_documents([search_text])
        results = self.db.retrieve(search_vectors, K, where=processed_filter) # , where=processed_filter
        
        idxs = [int(idx) for idx in results['ids'][0]]
        return pd.concat([pd.DataFrame({"distances": results['distances'][0]}, index=idxs), self.original_df.loc[idxs]], axis=1)

    
    @staticmethod
    def combine_context(relevant_contents): # Helper function for generating context
        context = relevant_contents.Content
        context = '\n'.join(context)

        return context
    
    def generate_context_prompt(self, search_text, K): # , where=None
        relevant_contents = self.retrieve_relevant(search_text=search_text, K=K)
        context = self.combine_context(relevant_contents=relevant_contents)
        prompt = self.prompt_template.format(context=context, search_text=search_text)
        return relevant_contents, prompt 

In [84]:
def df_db_pre(index_tb, tpwire_df):
    cols = ['dealname', 'propertysubtype', 'proptypecode', 'state', 'county', 'city', 'msaname', 'loanname', 'region', 'publishdate']
    df_data = tpwire_df[cols]
    
    return pd.concat([index_tb, df_data], axis=1)

# Execution

In [18]:
# def main(input_question):
#     import os
#     from langchain_openai import OpenAIEmbeddings
#     import chromadb
#     import chromadb.utils.embedding_functions as embedding_functions
#     from chromadb.config import Settings
#     from utils import date_to_int, convert_values, process_llm_output
#     from utils import SecretManager, OpenAIClient
#     from rag_utils import TpwireDataLoader, IndexDoc, tpwireDB, tpwireRAG, df_db_pre
    
    
#     # API key setting
#     SECRET_NAME= "AmazonSageMaker-sagemarker_yuwen"
#     secret_manager = SecretManager(secret_name=SECRET_NAME)
#     os.environ['OPENAI_API_KEY'] = secret_manager.get_secret('OPENAI_API_KEY')

#     # Set constant variable
#     BUCKET_NAME, PREFIX = "trepp-developmentservices-datascience/", "llm-exploration/treppwire_rag/"

#     MODEL = 'gpt-4o-mini'
#     DBNAME = 'treppwireRAG'
#     EMBED_MODEL_NAME = 'text-embedding-ada-002'
#     K = 5 # Number of relevant chunks


#     embed = OpenAIEmbeddings(model=EMBED_MODEL_NAME) # Create embeddings for indexing documents
#     openai_ef = embedding_functions.OpenAIEmbeddingFunction(
#         api_key=os.environ['OPENAI_API_KEY'], 
#         model_name=EMBED_MODEL_NAME)
    
    
#     # class initialization
#     openai_client = OpenAIClient(model=MODEL)
#     db_client = chromadb.PersistentClient(path='./treppwire-chromaDB')

#     # S3 parquet loader
#     s3_parquet_loader = S3ParquetLoader(bucket_name=BUCKET_NAME, prefix=PREFIX)

#     # Import TreppWire data
#     tpwire_loader = TpwireDataLoader(bucket_name=BUCKET_NAME, prefix=PREFIX)
    
    
#     # Index_Doc = IndexDoc(embed)
#     tpwire_df = tpwire_loader.process_data("tpwire_flags.parquet")
#     tpwire_df.head()
    
#     index_tb = s3_parquet_loader.load_s3_parquet('treppwire_index_tb.parquet')
#     original_df = df_db_pre(index_tb, tpwire_df)
    
#     # VectorDB
#     # tpwire_DB.delete()
#     # tpwire_DB.collection_exists()
#     tpwire_DB = tpwireDB(db_client, DBNAME, openai_ef)
#     # tpwire_DB.add(original_df=original_df)
    
    
#     # Initialize treppwire RAG
#     tpwire_RAG = tpwireRAG(tpwire_DB, original_df, embed)
#     search_text = input_question # 'What are the 5 transactions in New York in Dec, 2023?'
#     # search_text = 'How many properties in New York City are currently facing foreclosure?'

#     relevant_contents, prompt = tpwire_RAG.generate_context_prompt(search_text=search_text, K=K)
#     response = openai_client.get_completion(message=prompt)
#     return json.loads(response.choices[0].message.content)['Answer']

In [1]:
import os, json
from langchain_openai import OpenAIEmbeddings
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
from chromadb.config import Settings
from utils import date_to_int, convert_values, process_llm_output
from utils import SecretManager, OpenAIClient, S3ParquetLoader
from utils_rag import TpwireDataLoader, IndexDoc, tpwireDB, tpwireRAG, df_db_pre
from config import SECRET_NAME, BUCKET_NAME, PREFIX, MODEL, DBNAME, EMBED_MODEL_NAME, K

In [4]:
if __name__ == "__main__":
    secret_manager = SecretManager(secret_name=SECRET_NAME)
    os.environ['OPENAI_API_KEY'] = secret_manager.get_secret('OPENAI_API_KEY')

    embed = OpenAIEmbeddings(model=EMBED_MODEL_NAME) # Create embeddings for indexing documents
    openai_ef = embedding_functions.OpenAIEmbeddingFunction(
        api_key=os.environ['OPENAI_API_KEY'], 
        model_name=EMBED_MODEL_NAME)
    
    
    # class initialization
    openai_client = OpenAIClient(model=MODEL)
    db_client = chromadb.PersistentClient(path='./treppwire-chromaDB')

    # S3 parquet loader
    s3_parquet_loader = S3ParquetLoader(bucket_name=BUCKET_NAME, prefix=PREFIX)

    # Import TreppWire data
    tpwire_loader = TpwireDataLoader(bucket_name=BUCKET_NAME, prefix=PREFIX)
    
    
    # Index_Doc = IndexDoc(embed)
    tpwire_df = tpwire_loader.process_data("tpwire_flags.parquet")
    tpwire_df.head()
    
    index_tb = s3_parquet_loader.load_s3_parquet('treppwire_index_tb.parquet')
    original_df = df_db_pre(index_tb, tpwire_df)
    
    # VectorDB
    # tpwire_DB.delete()
    # tpwire_DB.collection_exists()
    tpwire_DB = tpwireDB(db_client, DBNAME, openai_ef)
    # tpwire_DB.add(original_df=original_df)
    
    
    # Initialize treppwire RAG
    tpwire_RAG = tpwireRAG(tpwire_DB, original_df, openai_client, embed)
    # input_question = 'What are the 5 transactions in New York in Dec, 2023?'
    # input_question = 'How many properties in New York City are currently facing foreclosure?'
    input_question = 'What significant events have occurred for Chicago Hotel?'
    search_text = input_question # 'What are the 5 transactions in New York in Dec, 2023?'
    # search_text = 'How many properties in New York City are currently facing foreclosure?'

    relevant_contents, prompt = tpwire_RAG.generate_context_prompt(search_text=search_text, K=K)
    response = openai_client.get_completion(message=prompt)
    print(f"The answer is: {json.loads(response.choices[0].message.content)['Answer']}")

The answer is: [{'event': 'InterContinental Hotel Chicago loan paid off', 'amount': '$127.6 million', 'date': 'March 2021', 'DSCR': '-1.21x'}, {'event': 'W Chicago – City Center loan became 30 days delinquent', 'amount': '$75 million', 'date': 'February 2021', 'occupancy': '46%', 'DSCR': '-1.04x'}, {'event': 'Value of W Chicago – City Center loan cut', 'amount': '$73.6 million', 'date': 'August 2021', 'previous_value': '$167 million'}, {'event': 'W Chicago – City Center loan paid off ahead of balloon date', 'amount': '$75.6 million', 'date': 'March 2023', 'DSCR': 'below 1.0x'}, {'event': 'Hilton Suites Chicago Magnificent Mile loan value lowered', 'amount': '$69.9 million', 'date': 'Recent', 'current_value': '$57.1 million', 'occupancy': '18%', 'DSCR': '0.90x in 2019'}]


-------------------
# RAG Evaluation

## Reranking Evaluation

### Hallucination questions

In [244]:
eval_data1 = [
    'Which property behind the GSMS 2012-GC6 deal had the lowest occupancy rate in 2019?',
    'Could you provide an overview of for Rockefeller Center loan?',
    'What significant events have occurred for Chicago Hotel in 2024?',
    'What are the key updates about the sale of Rockefeller Center in 2024?',
    'What are the highlights of credit story for North Carolina medical office building in 2018?',
    'Could you summarize the deal that property X belongs to?',
    'What changes in valuation have been recorded for Hmart Mall?',
    'Could you summarize the main details of transactions of New York Central Park?',
    'How was the delinquency status of Empire State Building been addressed in the latest update?',
    'Could you provide the details about world trade center sent to special servicing?',
    'What was the loss severity percentage for the Jacksonville retail property loan resolution this month?',
    'What impact does the transition of property to the lender have on the financial health of Virginia Mall Loan?',
    'What modifications were made to the loan for 11 West 42nd Street property in June 2018?',
    'What are the main factors contributing to the imminent default of the Fifth Avenue loan in Washington, DC?',
    'What was the outcome of the bankruptcy auction for the Lower Manhattan mixed-use property at 78 Bowery Street?',
    'What was the resolution outcome for the Rockefeller Center loan in New York, NY?',
    'What impact did the departure of Clock Tower Village have on the Fox River Mall loan?',
    'What are the implications of the recent tenant changes at One Newark Center for the Cupertino Hilton Hotel Loan?',
    'How has the value of Crossgates Mall loan evolved this year?',
    'What was the final outstanding balance loss percentage for the Ozarks Hotel property asset resolution?',
]

eval_data2 = [
    'What is the current delinquency rate for lodging loans in the CMBS market?',
    'What was the recent sale price of Property X?',
    'When was Avalon Riverview, Long island city constructed, and has it undergone any renovations?',
    "What's the average age of office buildings sold this quarter?",
    'What is the CMBX series that the Dartmouth Mall loan is part of?',
    'What is the current occupancy rate of the Empire State Building in New York, NY?',
    'Has there been any recent news or updates regarding the sale of the 55 Green Street property?',
    'Has the loan behind the North Carolina medical office building been refinanced recently?',
    'What is the current vacancy rate of the Blackwell Plaza property?',
    'What is the recent DSCR of the $108.8 million Embassy Suites loan backing the JPMCC 2019-EMBS deal in Manhattan?',
    'What caused the significant drop in occupancy rate at Jackson Park in Long Island City from 96% in 2019 to 59% in September 2018?',
    "What's the LTV of Gotham Point property in Queens?",
    'What percentage of deal K does the loan for property P represent?',
    'Who was the buyer in the recent transaction of Rockefeller Center?',
    'What is the current outstanding balance of the 55 Broadway loan?',
    'What was the DSCR (NCF) for Crossgates Mall in 2021?',
    'What is the likelihood of Schwab subletting the space at 211 Main Street in San Francisco?',
    'How has the valuation of Property X changed over the past few years?',
    "What's the location and size of property X?",
    'How many square feet does 55 Broadway loan have?',
]

eval_data3 = [
    'What are the 5 transactions in New York in this quarter?',
    'Show me all properties built before 1975 that were sold in the last quarter.',
    'How many multifamily communities were sold in Las Vegas in 2018?',
    'List 5 properties undergoning renovation in the previous quarter?',
    'List the 3 most recent transactions for properties larger than X square feet in State G.',
    'List the 5 largest loans (by dollar amount) that were sent to special servicing in New York this quarter.',
    'What is the average time on market for Property Type Multifamily sold in City U in the last 6 months?',
    'What is the average time on market for Property Type Multifamily sold in New York in the last 6 months?',
    'What was the occupancy rate of The Clusters apartment community at 4416 Northcrest Drive in Midland, TX in 2019?',
    'What properties in State NY sold for more than $10 million and less than $100 million this year?',
]

#### ---- eval_data1 ----

In [165]:
res = []
for q in eval_data1:
    relevant_contents, prompt  = tpwire_RAG.generate_context_prompt(search_text=q, K1=K1, K2=K2)
    response = openai_client.get_completion(message=prompt)
    res.append(json.loads(response.choices[0].message.content)['Answer'])

In [168]:
[int(i=="I don't know the answer") for i in res]

[1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1]

#### ---- eval_data2 ----

In [243]:
res = []
for q in eval_data2:
    relevant_contents, prompt = tpwire_RAG.generate_context_prompt(search_text=q, K1=K1, K2=K2)
    response = openai_client.get_completion(message=prompt)
    result = json.loads(response.choices[0].message.content)['Answer']
    res.append(result)

#### ---- eval_data3 ----

In [ ]:
res = []
for q in eval_data3:
    relevant_contents, prompt = tpwire_RAG.generate_context_prompt(search_text=q, K1=K1, K2=K2)
    response = openai_client.get_completion(message=prompt)
    result = json.loads(response.choices[0].message.content)['Answer']
    res.append(result)

In [247]:
[int(i=="I don't know the answer") for i in res]

[1, 0, 0, 1, 1, 0, 1, 1, 1, 0]

### Retrieval Questions

In [27]:
eval_d1 = [
    'What is the most recent status of Big Philly Office Loan?',
    'What are the key updates about the sale of Dulaney Center?',
    'Can you provide an overview of for National Business Park loan?',
    'How was the occupancy rate for Doner Company in the recent report?',
    'Can you summarize the recent financial performance of Chicago Hotel?',
    'What significant events have occurred for Chicago Hotel (after 2022)?',
    'How has the value of Waterfront at Port Chester loanevolved according to the latest credit report?',
    'What are the main points of concern for IDS Center loan mentioned (this month)?',
    'Can you detail the (recent) transactions involving One Concords Center?',
    'What are the highlights of the latest credit story for WeWork?',
    'How was the delinquency status of Aviation Mall been addressed in the latest update?',
    'What new information is available about the refinancing of New York Hospitality Portfolio loan?',
    'Could you provide a brief on the recent sales details of Courtyard Louisville Airport?',
    'What changes in valuation have been recorded for North Riverside Park Mall (this month)?',
    'How does the latest credit story describe the financial health of SASB Office Loan?',
    'Could you summarize the deal that Hughes Center Las Vegas belongs to?',
    'Could you summarize the main details of sales of Ozarks Hotel?',
    'Could you summarize the loan details of Ozarks Hotel?',
    'What are the highlights of the special servicing history for Cupertino Hilton Hotel Loan?',
    'Could you summarize the valuation changes for Cranberry Woods Office Park (over the years)?',
]


eval_d3 = [
    'What are the 5 transactions in New York in Dec, 2023?',
    'Can you list properties involved in deal FREMF 2019-K92?',
    'Could you list all stories about office properties in Houston, TX after 2020?',
    'What Convenience Center properties are in the Los Angeles-Long Beach-Anaheim, CA msa area?',
    'What loans in the MAD 2019-650M are in New York?',
    'What was the Bank of America Plaza Loan DSCR in 2022?',
    'What is the total Loan Per Square Foot (LPSF) for Bank of America Plaza in St. Louis?',
    'What was the occupancy rate increase of Bank of America Plaza in St. Louis from 2021 to 2022?',
    'What is the total  lease length of the Jamaica Center Cinema in Queens, NY?',
    'Has the East Village Multifamily Portfolio Pool 2 loan LTV increased since securitized? If so, how much of the increase.',
]

#### ---- eval_data1 ----

In [38]:
relevant_contents1, relevant_contents, prompt = tpwire_RAG.generate_context_prompt(search_text=eval_d1[1], K1=K1, K2=K2)

In [40]:
relevant_contents1

,distances,Content,Vectors,dealname,propertysubtype,proptypecode,state,county,city,msaname,loanname,region,publishdate
2606,0.377613,Maryland Retail Property: Joann Leaving; Aldi ...,"[-0.011157580651342869, -0.011717766523361206,...",[GSMS 2017-GS7],[Mixed Use],[MU],[NY],[New York],[New York],"[New York-Newark-Jersey City, NY-NJ-PA]",[90 Fifth Avenue],[Middle Atlantic],2023-01-09
1090,0.386609,GA Retail Property SoldShopping Center Busines...,"[-0.012449325993657112, -0.01281984243541956, ...",[COMM 2013-CR9],[Full Service],[LO],[TN],[Davidson],[Nashville],"[Nashville-Davidson--Murfreesboro--Franklin, TN]",[Hilton Nashville],[East South Central],2020-12-28
674,0.386949,Follow-up: Starwood National Mall PortfolioWe ...,"[-0.01682840660214424, 0.0006534524727612734, ...",[GCCFC 2006-GG7],[Urban Office],[OF],[TX],[Dallas],[Farmers Branch],"[Dallas-Fort Worth-Arlington, TX]",[JP Morgan International Plaza I & II - A-2 note],[West South Central],2020-04-03
2410,0.390775,Big Lease Renewal for Retail Property Behind 2...,"[-0.00617832038551569, -0.007162086199969053, ...",[BDS 2019-FL4],[Neighborhood Center],[RT],[NY],[Richmond],[Staten Island],"[New York-Newark-Jersey City, NY-NJ-PA]",[Shops at Richmond Plaza],[Middle Atlantic],2022-10-06
1467,0.393591,Short-Term Lease Renewal for Top Tenant Behind...,"[-0.0233136098831892, -0.013446380384266376, 0...",[BANK 2018-BN13],[Urban Office],[OF],[NY],[New York],[New York],"[New York-Newark-Jersey City, NY-NJ-PA]",[Ditson Building],[Middle Atlantic],2021-08-03
937,0.396283,Quick Hit: Lease Renewal Signed for Retail Pro...,"[-0.012666475959122181, -0.01589141972362995, ...",[MSBAM 2014-C15],[Regional Mall],[RT],[NY],[Albany],[Colonie],"[Albany-Schenectady-Troy, NY]",[Northway Mall],[Middle Atlantic],2020-09-02
13,0.400396,Macy’s to Close Store at DC-Area Mall Backing ...,"[-0.03777579590678215, -0.014440495520830154, ...",[BBCMS 2013-TYSN],[Regional Mall],[RT],[VA],[Fairfax],[Mclean],"[Washington-Arlington-Alexandria, DC-VA-MD-WV]",[Tysons Galleria Mall],[South Atlantic],2019-01-11
2756,0.401013,Quick Hit: 2014 Mall Portfolio to Lose Another...,"[-0.020021585747599602, -0.006417350843548775,...",[CLNC 2019-FL1],[Urban Office],[OF],[NY],[Queens],[Long Island City],"[New York-Newark-Jersey City, NY-NJ-PA]",[The Blanchard Building],[Middle Atlantic],2023-03-06
2734,0.401503,Florida Office Behind 2013 Loan SoldAccording ...,"[-0.02217288501560688, -0.020210441201925278, ...",[JPMBB 2013-C12],[Urban Office],[OF],[FL],[Hillsborough],[Tampa],"[Tampa-St. Petersburg-Clearwater, FL]",[Bridgeport Center],[South Atlantic],2023-02-27
1398,0.401586,Single-Tenant Industrial Property Behind 2017 ...,"[-0.005967846140265465, -0.021376958116889, 0....",[CD 2017-CD6],[Warehouse / Distribution],[IN],[NC],[Durham],[Durham],"[Durham-Chapel Hill, NC]",[FedEx Ground - Durham],[South Atlantic],2021-06-25


In [251]:
res = []
for q in eval_d1:
    relevant_contents, prompt = tpwire_RAG.generate_context_prompt(search_text=q, K1=K1, K2=K2)
    res.append(relevant_contents.index.tolist())

In [252]:
res

[[3332, 3341, 3539, 2786, 2585],
 [13, 2606, 1467, 937, 1090],
 [1046, 235, 1332, 2711, 879],
 [1215, 3071, 2778, 1206, 3502],
 [1546, 3080, 2512, 1440, 1563],
 [3080, 1181, 1518, 1132, 1896],
 [2520, 2995, 3181, 1125, 977],
 [2790, 2925, 3177, 2004, 198],
 [2043, 2766, 3206, 2431, 559],
 [3489, 659, 677, 3041, 3150],
 [3452, 2090, 797],
 [3432, 1131, 942, 2527, 2779],
 [1777, 2123, 3220],
 [2847, 2911, 2184, 3363, 3533],
 [3537, 2849, 3268, 2558, 2089],
 [1277, 196, 1321, 354, 2565],
 [3546, 3315, 1765, 1175, 779],
 [3546, 178, 2355, 2309, 1288],
 [3614, 3440, 2390, 1281, 1958],
 [3551, 1356, 3154, 3509, 2804]]

#### ---- eval_data3 ----

In [255]:
res = []
for q in eval_d3:
    relevant_contents, prompt = tpwire_RAG.generate_context_prompt(search_text=q, K1=K1, K2=K2)
    res.append(relevant_contents.index.tolist())

In [256]:
res

[[3602, 3569, 3115, 1991, 2059],
 [1848, 1351, 77, 1741, 271],
 [399, 819, 2416, 2597, 256],
 [590, 2975, 626, 52, 41],
 [323, 1021, 2080, 2089, 2558],
 [2979, 1921, 913, 3253, 3518],
 [2979, 3198, 1921, 55, 1851],
 [2979, 3198, 1921, 55, 1920],
 [3526, 2950, 2548, 2748, 2956],
 [2275, 1344, 3353, 1003, 2520]]

In [258]:
tpwire_df.iloc[1848].summaryplaintext

'Multifamily Property in Houston AcquiredThe Fuse at Park Row, a 318-unit multifamily property at 14220 Park Row Drive in Houston, TX, was acquired by Keener Investments, according to Multi-Housing News.The property backs the $28.2 million 14220 At Park Row loan which makes up 4.25% of FREMF 2019-KC07.The complex was constructed in 1998 and renovated in 2015.The loan matures in 2026 and is locked out until late 2025. (Yes, the loan has an unusually long 25-month open window.)For the first nine months of 2021, the loan posted a DSCR (NCF) of 2.00x when occupancy was 96%.The complex was valued at $41.17 million in 2019. The sales price was not disclosed.'

# * Metadata filtering exploration

## The properties I think will be useful is "proptypecode", "state, county, city" and "publishdate". But I put all tags into it.

In [29]:
tpwire_df.iloc[2760].summaryplaintext

'Trading Alert: Stop &amp; Shop Portfolio SoldFour Stop &amp; Shop properties in Massachusetts and Rhode Island were acquired by Orion Real Estate Group, as reported in the Providence Business First.The properties back a big 2020 CMBS loan that can be prepaid with a small yield maintenance charge. The four properties back the $45 million Stop &amp; Shop Portfolio loan which makes up 2.77% of BANK 2020-BN25.The loan matures in 2030 and it carries an interest rate of just 3.445%. If prepaid (and not assumed) the yield maintenance charge on the loan would be its “floor” of just 1% of the loan balance.The individual sales prices are as follows:'

In [24]:
db_client.get_collection(name=DBNAME).get(ids='2760')

{'ids': ['2760'],
 'embeddings': None,
 'metadatas': [{'publishdate': 1678233600}],
 'documents': [None],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [31]:
idx = tpwire_df['state'].value_counts().index

In [32]:
idx[100][1:-1].split(',')

['VR', ' MA', ' RI']

In [33]:
tpwire_df.state.tolist().index('[VR, MA, RI]')

2760

In [34]:
# states

In [35]:
metadata_prompt = """
    Could you using the Name Entity Recoginization for the below sentence and return the filter in the chromaDB format for me?
    There are 3 types of metadata:
        1. proptypecode: it contains values: {{OF, RT, LO, MF, MU, IN, OT, MU, MH}}
        2. state, county, city: it actually contains 3 types of metadata: state, county and city. You can ignore if the sentence doesn't have it, but if it does have that, you should figure them all out.
        3. publishdate: I want you to first learn about time-related keywords, so if keywords is something like "recent", "latest", just assume within one month, if keywords like "this quarter", use "start of this quarter to today".
    
    Input: 
    sentence: {sentence}
    
    Example:
    {{"$and":[
        {{
            "state": {{"$in": ["NY"]}}
            }}, 
        {{"publishdate": 
            {{
                "$gte": date_to_int("2024-01-01")
                }}
            }}
            ]
        }} 
    means in state NY and after date 2024-01-01.
    
    Output:
    <chromaDB standard filter>
"""

metadata_prompt1 = """
    Please use Named Entity Recognition (NER) to extract information from the following sentence and format the output according to the ChromaDB filter standard. (If "$gte" and "lte" appear together, you should output in individual dict)

    There are 5 types of metadata to consider:
        1. proptypecode: It can contain values such as {{OF, RT, LO, MF, MU, IN, OT, MU, MH}}.
        2. state: These are geographical entities. Extract all that are present in the sentence. If it is missing, ignore. (state should be 2 characters abbreviation)
        3. county: These are geographical entities. Extract all that are present in the sentence. If it is missing, ignore.
        4. city: These are geographical entities. Extract all that are present in the sentence. If it is missing, ignore. 
        5. publishdate: Identify time-related keywords. If the keywords are like "recent", "latest", assume a date within one month. If keywords are like "this quarter", use the range "start of this quarter to today". If keywords are like "in monthA", use the range "start of this month to today". (all dates should be in the standard yyyy-mm-dd format)

    Input:
    sentence: {sentence}
    
    Example:
    If the sentence is "Properties in NY have seen recent updates":
    {{
        "$and": [
            {{
                "state": {{"$in": ["NY"]}}
            }},
            {{
                "publishdate": {{
                    "$gte": "2024-06-01"
                }}
            }}
        ]
    }}

    Output:
    Provide the filter in the ChromaDB standard format.
"""

metadata_prompt2 = """
    Please use Named Entity Recognition (NER) to extract information from the following sentence and format the output according to the ChromaDB filter standard. 

    There are 5 types of metadata to consider:
        1. proptypecode: It can contain values such as {{OF (Office), RT (Retail), LO (Lodging), MF (Multi-family), MU (Multi-use), IN (Industrial), OT (Other), MH (Mobile home)}}. If doesn't have, ignore it.
        2. state: These are geographical entities. Extract all that are present in the sentence. If it is missing, ignore it. (state should be a 2-character abbreviation)
        3. county: These are geographical entities. Extract all that are present in the sentence. If it is missing, ignore it.
        4. city: These are geographical entities. Extract all that are present in the sentence. If it is missing, ignore it.
        5. publishdate: Identify time-related keywords. If the keywords are like "recent", "latest", assume a date within one month. If keywords are like "this quarter", use the range "start of this quarter to today". If keywords are like "in monthA", use the range "start of this month to today". (all dates should be in the standard yyyy-mm-dd format)

    If both "$gte" and "$lte" conditions appear, they should be output separately as individual dictionaries.

    Input:
    sentence: {sentence}
    
    Example:
    If the sentence is "Properties in NY have seen recent updates":
    {{
        "$and": [
            {{
                "state": {{"$in": ["NY"]}}
            }},
            {{
                "publishdate": {{
                    "$gte": "2024-06-01"
                }}
            }}
        ]
    }}

    If the sentence is "Properties in NY have seen updates in July, 2024":
    {{
        "$and": [
            {{
                "state": {{"$in": ["NY"]}}
            }},
            {{
                "publishdate": {{
                    "$gte": "2024-07-01"
                }}
            }},
            {{
                "publishdate": {{
                    "$lte": "2024-07-30"
                }}
            }}
        ]
    }}

    Output:
    Provide the filter in the ChromaDB standard format.
"""


metadata_prompt4 = """
    Please extract information from the given sentence and format it according to the ChromaDB filter standard. Follow these steps:

    1. Entity Extraction:
        - proptypecode: It can contain property type values such as {{OF (Office), RT (Retail), LO (Lodging), MF (Multi-family), MU (Multi-use), IN (Industrial), OT (Other), MH (Mobile home)}}. If doesn't contain, just ignore this.
        - state: These are geographical entities. Extract all that are present in the sentence. If it is missing, ignore it. (state should be a 2-letter code)
        - county: These are geographical entities. Extract all that are present in the sentence. If it is missing, ignore it.
        - city: These are geographical entities. Extract all that are present in the sentence. If it is missing, ignore it.
        - publishdate: Identify time-related keywords

    2. publishdate interpretation:
       - "recent" or "latest": Use date range from 30 days ago to today
       - "this quarter": Use range from start of current quarter to today
       - "in [month]": Use range from start to end of specified month
       - "in [month], [year]": Use range from start to end of specified month and year
       - Specific dates: Use as given
       - All dates should be should be in the standard yyyy-mm-dd format.

    3. Formatting:
       - Use the ChromaDB filter standard
       - For date ranges, use separate "$gte" and "$lte" conditions
    
    Examples:

    1. "Find office properties in New York City listed this quarter"
    {{
        "$and": [
            {{
                "proptypecode": {{"$in": ["OF"]}}
            }},
            {{
                "city": {{"$in": ["New York City"]}}
            }},
            {{
                "publishdate": {{"$gte": "2024-07-01"}}
            }},
            {{
                "publishdate": {{"$lte": "2024-07-31"}}
            }}
        ]
    }}

    2. "Show recent retail listings in Los Angeles County, CA"
    {{
        "$and": [
            {{
                "proptypecode": {{"$in": ["RT"]}}
            }},
            {{
                "state": {{"$in": ["CA"]}}
            }},
            {{
                "county": {{"$in": ["Los Angeles"]}}
            }},
            {{
                "publishdate": {{"$gte": "2024-07-01"}}
            }},
            {{
                "publishdate": {{"$lte": "2024-07-31"}}
            }}
        ]
    }}

    3. "What are the 5 transactions in New York in Dec, 2023?"
    {{
        "$and": [
            {{
                "state": {{"$in": ["NY"]}}
            }},
            {{
                "publishdate": {{"$gte": "2023-12-01"}}
            }},
            {{
                "publishdate": {{"$lte": "2023-12-31"}}
            }}
        ]
    }}

    Please provide the ChromaDB filter for the following sentence:
    Input sentence: {sentence}
"""

In [36]:
# metadata_prompt11 = metadata_prompt1.format(sentence="How many properties in the Houston, TX facing foreclosure?")
metadata_prompt41 = metadata_prompt4.format(sentence='What are the 5 transactions in New York in Dec, 2023?') 

In [ ]:
# tpwire_RAG.retrieve_relevant(search_text='What are the 5 transactions in New York in Dec, 2023?', K=K, where=processed_filter)
# search_text='What are the 5 transactions in New York in Dec, 2023?'
# search_text = 'What was the Bank of America Plaza Loan DSCR in 2022?'
# search_text = 'What is the total Loan Per Square Foot (LPSF) for Bank of America Plaza in St. Louis?'
# search_text = 'What was the occupancy rate increase of Bank of America Plaza in St. Louis from 2021 to 2022?'
search_text = "What's the total lease length of the Jamaica Center Cinema in Queens, NY?"

response = openai_client.get_completion(message= metadata_prompt4.format(sentence=search_text))
where_filter = json.loads(response.choices[0].message.content)
processed_filter = process_llm_output(where_filter)
relevant_contents, prompt = tpwire_RAG.generate_context_prompt(search_text=search_text, K=K, where=processed_filter)

In [ ]:
relevant_contents

In [ ]:
response = openai_client.get_completion(message=prompt)
json.loads(response.choices[0].message.content)['Answer']

In [ ]:
prompt

In [ ]:
tpwire_df.iloc[2760].summaryplaintext, tpwire_df.iloc[2760].state

('Trading Alert: Stop &amp; Shop Portfolio SoldFour Stop &amp; Shop properties in Massachusetts and Rhode Island were acquired by Orion Real Estate Group, as reported in the Providence Business First.The properties back a big 2020 CMBS loan that can be prepaid with a small yield maintenance charge. The four properties back the $45 million Stop &amp; Shop Portfolio loan which makes up 2.77% of BANK 2020-BN25.The loan matures in 2030 and it carries an interest rate of just 3.445%. If prepaid (and not assumed) the yield maintenance charge on the loan would be its “floor” of just 1% of the loan balance.The individual sales prices are as follows:',
 '[VR, MA, RI]')

# * Reranking exploration

## Using cross-encoder: pass out because of the limitation of context length.

In [45]:
from sentence_transformers import CrossEncoder

In [46]:
model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', max_length=512)

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [48]:
relevant_contents

,distances,Content,Vectors,dealname,propertysubtype,proptypecode,state,county,city,msaname,loanname,region,publishdate
3486,0.441708,Big NYC Office Loan Expected to Miss Maturity ...,"[-0.022681208327412605, -0.012724092230200768,...",[COMM 2014-CR15],[Urban Office],[OF],[NY],[New York],[New York],"[New York-Newark-Jersey City, NY-NJ-PA]",[25 West 45th Street],[Middle Atlantic],2023-12-14
3499,0.443006,Trading Alert: NYC’s Seagram Building Refinanc...,"[-0.020313233137130737, -0.02171507477760315, ...",[COMM 2013-CR8],[Mixed Use],[MU],[NY],[New York],[New York],"[New York-Newark-Jersey City, NY-NJ-PA]",[The Prince Building],[Middle Atlantic],2023-12-20
3498,0.453352,Another Concern for NYC Mixed-Use Property?A f...,"[-0.02200057916343212, 0.0014346742536872625, ...",[GSMS 2014-GC26],[Urban Office],[OF],[CA],[Los Angeles],[Los Angeles],"[Los Angeles-Long Beach-Anaheim, CA]",[Bank of America Plaza],[Pacific],2023-12-19
3511,0.454713,Correction: Connecticut Retail Loan Last week ...,"[-0.03223033621907234, 0.005310523323714733, 0...",[UBSBB 2013-C5],[Urban Office],[OF],[NY],[New York],[New York],"[New York-Newark-Jersey City, NY-NJ-PA]",[200 Park Avenue],[Middle Atlantic],2023-12-26
3489,0.455509,Trepp Flash: WeWork Renegotiates Lease for Big...,"[-0.011740509420633316, -0.016708938404917717,...",[JPMCC 2021-1440],[Mixed Use],[OF],[NY],[New York],[New York],"[New York-Newark-Jersey City, NY-NJ-PA]",[1440 Broadway],[Middle Atlantic],2023-12-15


In [49]:
search_text

'What are the 5 transactions in New York in Dec, 2023?'

In [59]:
lst, idxs = [], []
for i, row in relevant_contents.iterrows():
    lst.append((search_text, row['Content']))
    idxs.append(i)
    
scores = model.predict(lst)

In [62]:
scores_dict = {idxs[i]: scores[i] for i in range(len(idxs))}

In [65]:
sorted(scores_dict.items(), key=lambda x: x[1], reverse=True)

[(3486, -3.9826086),
 (3489, -4.3170896),
 (3498, -4.8581214),
 (3499, -9.419075),
 (3511, -11.082737)]

In [56]:
scores.sort(

array([ -3.9826086,  -9.419075 ,  -4.8581214, -11.082737 ,  -4.3170896],
      dtype=float32)

In [57]:
for i, row in relevant_contents.iterrows():
    print(i)

3486
3499
3498
3511
3489


In [67]:
from transformers import AutoTokenizer

In [79]:
tokenizer = AutoTokenizer.from_pretrained("cross-encoder/ms-marco-MiniLM-L-6-v2")
query = search_text
paragraph = relevant_contents.iloc[5].Content

tokens = tokenizer(query, paragraph, truncation=False)
token_length = len(tokens['input_ids'])

print(f"Token length: {token_length}")

IndexError: single positional indexer is out-of-bounds

In [84]:
tpwire_df.iloc[0].summaryplaintext

'Purchase of Seattle-Area Office FinalizedThe Registry reported that the Highlands Campus Tech Center, a 201,766 square-foot office complex located in Bothell, Washington has been sold. The property serves as collateral for the $23.4 million Highlands Campus Tech Center loan which represents 2.25% of the GSMS 2014-GC24 deal.DSCR (NCF) on the loan was 1.96x during 2017 and 2.29x for the first half of 2018. The latest occupancy reading was 97%.Per The Registry, the property was purchased for $33.6 million. The property was appraised for 33.9 million at securitization, giving it an LTV of 68.9% at the time.A prepayment lockout is in place until April 2021, three months prior to the loan’s maturity.'

In [85]:
num_of_token = []
query = search_text

for i, row in tpwire_df.iterrows():
    paragraph = row['summaryplaintext']
    tokens = tokenizer(query, paragraph, truncation=False)
    token_length = len(tokens['input_ids'])
    num_of_token.append(token_length)

Token indices sequence length is longer than the specified maximum sequence length for this model (524 > 512). Running this sequence through the model will result in indexing errors


## RankingGPT

In [106]:
ranking_prompt_template = """
    Given the following query and a list of commercial real estate credit stories, rank those stories in order of relevance to the query. Each document is represented by its index and content. The output should be a list of story index ordered by relevance, separated by a ">" token.

    The stories is a list of dicts and its format should be:
    [
        {{index1: story1}},
        {{index2: story2}},
        ...
        {{indexN: storyN}},
    ]
    
    Query: {query}
    Stories: {stories}
    

    Output the ranked stories index in the JSON format: 
    {{
        index1,
        index2,
        ...
        indexN,
    }}
    means index1 > index2 > index3 > ... > indexN
"""

In [91]:
stories = []
for i, row in relevant_contents.iterrows():
    stories.append({i: row['Content']})   

In [129]:
stories = [{i: row['Content']} for i, row in relevant_contents.iterrows()]
stories

[{3486: 'Big NYC Office Loan Expected to Miss Maturity Date, Heads to Special ServicingDecember servicer data shows that the $64.6 million 25 West 45th Street loan has been sent to special servicing. Remittance comments state the loan is not expected to be paid off at its maturity date next month. The collateral is a 188,365-square-foot office in Midtown Manhattan. For H1 2023, the loan posted a DSCR (NCF) of 1.46x when occupancy was 68%. Occupancy is down from 86% in 2020.The loan makes up 15.5% of the collateral behind COMM 2014-CR15. The property was valued at $107 million in 2014 giving the loan an LTV of 65 at the time. Since then, there has been more than $5.4 million in principal amortization.\xa0'},
 {3499: 'Trading Alert: NYC’s Seagram Building Refinanced(Editor’s note: The following story was circulated via a separate email alert Tuesday Afternoon. We are recirculating here for those who missed it.) The mortgage on the Seagram Building in Midtown Manhattan has been refinanced

In [107]:
ranking_prompt = ranking_prompt_template.format(query=search_text, stories=stories)

In [108]:
response = openai_client.get_completion(message=ranking_prompt)

{'index1': 3489,
 'index2': 3486,
 'index3': 3499,
 'index4': 3498,
 'index5': 3511}

In [126]:
k2 = 5
rank_index = list(json.loads(response.choices[0].message.content).values())[0:k2]

In [128]:
relevant_contents.loc[rank_index]

,distances,Content,Vectors,dealname,propertysubtype,proptypecode,state,county,city,msaname,loanname,region,publishdate
3489,0.455509,Trepp Flash: WeWork Renegotiates Lease for Big...,"[-0.011740509420633316, -0.016708938404917717,...",[JPMCC 2021-1440],[Mixed Use],[OF],[NY],[New York],[New York],"[New York-Newark-Jersey City, NY-NJ-PA]",[1440 Broadway],[Middle Atlantic],2023-12-15
3486,0.441708,Big NYC Office Loan Expected to Miss Maturity ...,"[-0.022681208327412605, -0.012724092230200768,...",[COMM 2014-CR15],[Urban Office],[OF],[NY],[New York],[New York],"[New York-Newark-Jersey City, NY-NJ-PA]",[25 West 45th Street],[Middle Atlantic],2023-12-14
3499,0.443006,Trading Alert: NYC’s Seagram Building Refinanc...,"[-0.020313233137130737, -0.02171507477760315, ...",[COMM 2013-CR8],[Mixed Use],[MU],[NY],[New York],[New York],"[New York-Newark-Jersey City, NY-NJ-PA]",[The Prince Building],[Middle Atlantic],2023-12-20
3498,0.453352,Another Concern for NYC Mixed-Use Property?A f...,"[-0.02200057916343212, 0.0014346742536872625, ...",[GSMS 2014-GC26],[Urban Office],[OF],[CA],[Los Angeles],[Los Angeles],"[Los Angeles-Long Beach-Anaheim, CA]",[Bank of America Plaza],[Pacific],2023-12-19
3511,0.454713,Correction: Connecticut Retail Loan Last week ...,"[-0.03223033621907234, 0.005310523323714733, 0...",[UBSBB 2013-C5],[Urban Office],[OF],[NY],[New York],[New York],"[New York-Newark-Jersey City, NY-NJ-PA]",[200 Park Avenue],[Middle Atlantic],2023-12-26


In [101]:
def num_tokens_from_string(string: str, model_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model(model_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

# Example usage
input_text = ranking_prompt
model_name = "gpt-4o-mini"
token_count = num_tokens_from_string(input_text, model_name)
print(f"Number of tokens: {token_count}")

Number of tokens: 162
